In [18]:
import pydicom
import cv2
import os
import dask
from tqdm.notebook import tqdm
from dask.diagnostics import ProgressBar
from pathlib import Path

In [2]:
data_dir = Path('./stage_2_images/')

dcm_files = list(data_dir.glob("*.dcm"))
len(dcm_files)

3205

In [3]:
outdir = "./jpg/"

if not os.path.exists(outdir):
    os.mkdir(outdir)

In [6]:
def dcm_to_jpg(filepath):
    ds = pydicom.read_file(str(filepath))
    img = ds.pixel_array

    dcm_file_name = filepath.parts[-1]
    jpg_file_name = dcm_file_name.replace('.dcm','.jpg')
    cv2.imwrite(outdir + jpg_file_name, img)

In [7]:
for f in tqdm(dcm_files):
    dcm_to_jpg(f)

In [20]:
@dask.delayed
def dcm_to_jpg_dask(filepath):
    ds = pydicom.read_file(str(filepath))
    img = ds.pixel_array

    dcm_file_name = filepath.parts[-1]
    jpg_file_name = dcm_file_name.replace('.dcm','.jpg')
    cv2.imwrite(outdir + jpg_file_name, img)

In [21]:
tasks = [dcm_to_jpg_dask(filepath) for filepath in dcm_files]
with ProgressBar():
    dask.compute(tasks)

[########################################] | 100% Completed | 19.6s
